# \__slots__ Magic

링크: https://github.com/nicewook/intermediatePython/blob/master/__slots__magic.rst

파이썬의 모든 클래스는 인스턴스 attributes 를 가질 수 있다. 

디폴트로 이런 attributes 를 dict 에 보관한다. 

이것은 런타임에서 임의로 새로운 attributes 를 설정하는데 큰 도움이 된다. 

<br>
하지만 이미 알고 있는 attributes 를 가진 작은 클래스에서는 오히려 bottleneck 이 된다. 

왜냐면 dict 는 메모리를 많이 먹기 때문이다. 

static 하게 dict 를 잡을 수 없기 때문에 수천 - 수백만 object 를 생성한다면 문제가 된다. 

이걸 피해가는 방법중 하나가 \__slots__ 를 사용하여 파이썬에게 dict 를 사용하지 않고, 

정해진 attributes set 에 대해서만 메모리를 할당하겠다고 하는 것이다. 




## 예제를 보자 

### \__slots__ 를 사용하지 않는 경우

In [29]:
class MyClass(object):
    
    def __init__(self, name, identifier):
            self.name = name
            self.identifier = identifier
            self.set_up()
    
    # ...

In [29] used 0.0078 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 113.28 MiB
In [29] used 0.0039 MiB RAM in 0.20s, peaked 0.00 MiB above current, total RAM usage 113.28 MiB


### \__slots__ 를 사용하는 경우

In [30]:
class MyClass(object):
    
    __slots__ = ['name', 'identifier']
    
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        self.set_up()
        
    # ...

In [30] used 0.0195 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 113.30 MiB
In [30] used 0.0000 MiB RAM in 0.20s, peaked 0.00 MiB above current, total RAM usage 113.30 MiB


\__slots__ 를 사용하는 두번째 코드는 RAM 부담을 줄여줄 것이다.

어떤 사람은 40 - 50% 효과를 보았다 한다. 

(참고) PyPy를 써보는 것도 좋다. 이런 형태의 최적화를 default 로 해준다.



## 아래 예제를 통해 실제로 얼마나 메모리를 사용하는지 확인 할 수 있다. 

https://github.com/ianozsvald/ipython_memory_usage

In [31]:
# 인스톨이 필요함
!pip install ipython_memory_usage -q

distributed 1.21.8 requires msgpack, which is not installed.


In [31] used 0.0039 MiB RAM in 2.08s, peaked 0.00 MiB above current, total RAM usage 113.30 MiB
In [31] used 0.0000 MiB RAM in 2.18s, peaked 0.00 MiB above current, total RAM usage 113.30 MiB


In [32]:
import ipython_memory_usage.ipython_memory_usage as imu

In [32] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 113.30 MiB
In [32] used 0.0000 MiB RAM in 0.20s, peaked 0.00 MiB above current, total RAM usage 113.30 MiB


In [33]:
imu.start_watching_memory()

In [33] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 113.30 MiB
In [33] used 0.0000 MiB RAM in 0.20s, peaked 0.00 MiB above current, total RAM usage 113.30 MiB
In [33] used -0.0234 MiB RAM in 0.30s, peaked 0.02 MiB above current, total RAM usage 113.28 MiB


In [34]:
!type slots.py

class MyClass(object):
        __slots__ = ['name', 'identifier']
        def __init__(self, name, identifier):
                self.name = name
                self.identifier = identifier

num = 1024*256
x = [MyClass(1,1) for i in range(num)]
In [34] used 0.0547 MiB RAM in 0.32s, peaked 0.00 MiB above current, total RAM usage 113.34 MiB
In [34] used 0.0000 MiB RAM in 0.42s, peaked 0.00 MiB above current, total RAM usage 113.34 MiB
In [34] used -0.0781 MiB RAM in 0.52s, peaked 0.08 MiB above current, total RAM usage 113.26 MiB


In [35]:
# __slots__ 를 사용하면 메모리 사용이 적다
from slots import *

In [35] used 0.0781 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 113.34 MiB
In [35] used 0.0000 MiB RAM in 0.20s, peaked 0.00 MiB above current, total RAM usage 113.34 MiB
In [35] used -0.0234 MiB RAM in 0.30s, peaked 0.02 MiB above current, total RAM usage 113.31 MiB


In [36]:
!type noslots.py

class MyClass(object):
        def __init__(self, name, identifier):
                self.name = name
                self.identifier = identifier

num = 1024*256
x = [MyClass(1,1) for i in range(num)]
In [36] used 0.0273 MiB RAM in 0.24s, peaked 0.00 MiB above current, total RAM usage 113.34 MiB
In [36] used 0.0000 MiB RAM in 0.34s, peaked 0.00 MiB above current, total RAM usage 113.34 MiB
In [36] used -0.0820 MiB RAM in 0.44s, peaked 0.08 MiB above current, total RAM usage 113.26 MiB


In [37]:
# __slots__ 를 사용하지 않으면 메모리 사용량이 많다.
from noslots import *

In [37] used 0.0781 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 113.34 MiB
In [37] used 0.0000 MiB RAM in 0.20s, peaked 0.00 MiB above current, total RAM usage 113.34 MiB
In [37] used -0.0234 MiB RAM in 0.30s, peaked 0.02 MiB above current, total RAM usage 113.31 MiB
